# Debt Market Model

## **Out of date.** See `simulations/` directory for specific simulation notebooks and results analysis notebooks.

$$
\Delta{t} = t_{k+1} - t_{k}\\
{Q}_{k+1} = {Q}_k + v_1 - v_2 - v_3\\
{D_1}_{k+1} = {D_1}_k + u_1 - u_2 - u_3\\
w_3 = u_3 \cdot \frac{w_2}{u_2}\\
w_1 = [(1+\beta_k)^{\Delta{t}}-1]({D_1}_k+{D_2}_k)\\
{D_2}_{k+1} = {D_2}_k + w_1 - w_2 - w_3\\
{R}_{k+1} = {R}_k + w_2\\
$$

<center>
<img src="./diagrams/debt_dynamics.png"
     alt="Debt dynamics"
     style="width: 60%" />
</center>

<center>
<img src="./diagrams/apt_model.png"
     alt="APT model"
     style="width: 60%" />
</center>

## First phase
* Debt market state -> ETH price changes (exogenous) -> exogenous u,v -> endogenous w -> mutates system state

## Second phase
* APT model, arbitragers act -> u,v activity (to remove diversifiable risk) -> results in change to both debt market and secondary market -> stability controller updates redemption rate and price

## Current Model

1. Historically driven ETH price, locks, and draws (eventually to be driven by APT model)
2. Endogenous liquidation and closing of CDPs
3. Debt market state

# Notes

## Resources
* https://github.com/BlockScience/reflexer/blob/next-steps/next_steps.MD
* https://community-development.makerdao.com/en/learn/vaults/liquidation/

* Close CDPs along debt age distribution around 3 months
* How many CDPs are opened daily?
* How are CDPs closed?
* Assumption: opened vs. topped up CDP e.g. ETH price drops, v1 + u1 increase
  * Rate of change of ETH price, make better assumption about new CDP vs top up
  * Break down daily v1/u1 data into multiple CDPs/top ups based on assumption
  * Extreme events -> indicates top up of existing CDP (one that's fallen below certain liquidation ratio)
  
* Large to small CDP liquidation: 50/50 - 2000/1000 at start of 2019
* 1000 to 2000 active CDPs
* 300% average collat ratio

See [Maker network report](https://www.placeholder.vc/blog/2019/3/1/maker-network-report)

> Towards the end of 2018,collateralization   spiked   to   nearly   400%, perhaps  due  to  heightened  risk-aversion  on the  part  of  CDP  holders,but  has  recently declined  back  to  ~270%,  slightly  under  the system’s average of ~300%.

> As  shown  in  Figure  2A, the average non-empty  CDP  declined  from  above $60Kdaiin  debtat  the  start  of  2018  to  just  over $30Kat  the  start  of  2019.Meanwhile,  the medianCDPby debtgrew from under $500in  debtat  the  start  of  the  year,  reaching around $4Kin   August,before   declining sharply to around$500by early February.

> The  significant delta between  mean and  mediandebts highlights thepower  law distribution acrossCDPs. While small CDPs dominate  by number—with  over  80%  of CDPsdrawingless  than $10K  of  dai—they representjust  over 3%  oftotal debt  in  the system.  On  the  other  end  of  the  spectrum, about 90CDPs (less  than  4%by  number) individually have  more  than $100Kin  dai outstanding,  representing nearly  84%  of  all debt  in  the  system.

> Such concentrationin     debtcan     be problematicfor dai supply.For example, four of the six periodsof dai contractiondiscussed in the previous section were associated with CDPs that  had  over$500K  in  debtbeing liquidated. For  example,  CDP  614 hadover 4.3  million in  debt at  liquidation  on  March 18th, accountingfor much of the contraction in outstanding     dai at     the     time. More dramatically,  the  liquidation  of CDPs  3228 and   3164,on   November   20thand   25threspectively,amounted  to  a  contraction  of over $10.7M in dai, making these two CDPs the primary culprits of thelargest contraction in   daisupplyof2018(i.e.   mid-to-late November as showninFigure 1B).

# Imports

In [1]:
#%pip install git+https://github.com/danlessa/cadCAD@no_deepcopy

In [2]:
#%pip show cadCAD

In [3]:
from shared import *

In [4]:
import numpy as np
import datetime as dt
import pandas as pd

# Historic MakerDAO Dai debt market activity

In [5]:
debt_market_df = pd.read_csv('market_model/data/debt_market_df.csv', index_col='date', parse_dates=True)
debt_market_df

,rho_star,beta,p,Q,v_1,v_2 + v_3,u_1,u_2 + u_3,u_2,w_2,u_3,w_3,w_2 + w_3,D_1,D_2,w_1,D,C_star,p_star,p_ema_10
date,,,,,,,,,,,,,,,,,,,,
2017-12-18,736.004090,0.005,1.017605,2.425662e+03,2526.662000,101.000000,5.953420e+05,4.594384e+01,1.000000e+01,0.000006,35.943840,0.000023,0.000029,5.952961e+05,0.000000e+00,0.000000,5.952961e+05,1.785297e+06,1,1.008573
2017-12-19,832.236972,0.005,1.003376,6.240431e+03,3821.320309,6.550995,1.379832e+06,3.539448e+04,3.536748e+04,0.455885,27.000000,0.000348,0.456233,1.939733e+06,2.604943e+01,26.505662,1.939759e+06,5.193518e+06,1,1.008573
2017-12-20,810.287507,0.005,1.012972,1.418307e+04,8066.895210,124.255903,2.676383e+06,5.022518e+05,5.014128e+05,0.178879,839.000000,0.000299,0.179179,4.113865e+06,8.208488e+01,56.214632,4.113947e+06,1.149236e+07,1,1.008573
2017-12-21,837.231080,0.005,1.025738,1.564473e+04,1471.742069,10.080000,8.053550e+05,4.978271e+04,4.978271e+04,1.158909,0.000000,0.000000,1.158909,4.869437e+06,1.474660e+02,66.539983,4.869584e+06,1.309826e+07,1,1.008573
2017-12-22,689.014990,0.005,0.972228,1.883084e+04,5316.960481,2130.848985,3.145512e+05,7.474531e+05,7.239153e+05,19.756468,23537.818000,0.642374,20.398842,4.436535e+06,1.876926e+02,60.625449,4.436723e+06,1.297473e+07,1,1.008573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-26,167.637314,0.125,1.021405,1.525944e+06,56696.770732,29566.578527,8.079170e+05,1.585480e+06,1.515115e+06,64257.311824,70364.380268,2984.212338,67241.524162,7.957442e+07,4.562428e+06,27167.708945,8.413685e+07,2.558052e+08,1,1.008076
2019-09-27,167.347085,0.125,1.007346,1.529959e+06,7598.158072,3582.812664,6.087607e+05,6.367461e+05,6.366783e+05,7214.430099,67.790257,0.768156,7215.198255,7.954644e+07,4.582358e+06,27145.743151,8.412880e+07,2.560342e+08,1,1.009903
2019-09-28,174.289775,0.125,1.011006,1.528450e+06,876.961579,2386.452089,2.251956e+05,1.452312e+05,1.452062e+05,492.589362,25.000000,0.084809,492.674170,7.962640e+07,4.609044e+06,27177.983765,8.423545e+07,2.663932e+08,1,1.011184


In [6]:
debt_market_df['u_2'].describe()

count    6.520000e+02
mean     4.204655e+05
std      7.017645e+05
min      0.000000e+00
25%      4.694582e+04
50%      1.727243e+05
75%      4.848118e+05
max      6.270788e+06
Name: u_2, dtype: float64

In [7]:
debt_market_df.insert(0, 'seconds_passed', 24 * 3600)
debt_market_df['cumulative_v_1'] = debt_market_df['v_1'].cumsum()

In [8]:
debt_market_df.plot()

# APT Model Setup

In [9]:
features = ['beta', 'Q', 'v_1', 'v_2 + v_3', 
                    'D_1', 'u_1', 'u_2', 'u_3', 'u_2 + u_3', 
                    'D_2', 'w_1', 'w_2', 'w_3', 'w_2 + w_3',
                    'D']
features_ml = ['beta', 'Q', 'v_1', 'v_2 + v_3', 'u_1', 'u_2', 'u_3', 'w_1', 'w_2', 'w_3', 'D']
optvars = ['u_1', 'u_2', 'v_1', 'v_2 + v_3']

start_date = '2018-11-05'

historical_initial_state = {k: debt_market_df.loc[start_date][k] for k in features}
historical_initial_state

{'beta': 0.025,
 'Q': 947045.6487308872,
 'v_1': 2857.187535228906,
 'v_2 + v_3': 1000.5056975099724,
 'D_1': 68492685.99189329,
 'u_1': 464779.30977171654,
 'u_2': 84787.98283784003,
 'u_3': 8.87666828930378e-10,
 'u_2 + u_3': 84787.98283784091,
 'D_2': 290170.723888923,
 'w_1': 4653.072700785418,
 'w_2': 22.312545277889768,
 'w_3': 2.335956777043483e-13,
 'w_2 + w_3': 22.312545277890006,
 'D': 68782856.71578223}

## Root function

In [10]:
import pickle

model = pickle.load(open('models/pickles/apt_debt_model_2020-11-28.pickle', 'rb'))

# ML debt model root function
def G(x, to_opt, data, constant):
    for i,y in enumerate(x):
        data[:,to_opt[i]] = y
    err = model.predict(data)[0] - constant
    return err

dpres = pickle.load(open('models/pickles/debt_market_OLS_model.pickle', 'rb'))

def G_OLS(x, to_opt, data, constant):
    for i,y in enumerate(x):
        data[:,to_opt[i]] = y
    err = dpres.predict(data)[0] - constant
    #print(f'G_OLS err: {err}')
    return err

ml_data_list = []
global tol
tol = 1e-2
global curr_error, best_error, best_val
global strikes
strikes = 0
best_error = 1e10

def glf_continue_callback(xopt):
    print('entered callback')
    global curr_error, best_error, best_val, strikes, tol
    if curr_error > tol: # keep searching
        print('bigger than tol, keep searching')
        return False
    else:
        if curr_error > best_error: # add strike
            strikes += 1
            if strikes < 3: # continue trying
                print('bigger than prev best, add strike')
                return False
            else: # move on, not working
                strikes = 0
                print('3rd strike, stop')
                return True
        else: # better outcome, continue
            best_error = curr_error
            best_val = xopt
            strikes = 0
            print('New best, reset strikes')
            return False

# Global minimizer function
def glf(x, to_opt, data, constant, timestep):
    global curr_error
    for i,y in enumerate(x):
        #print(x)
        data[:,to_opt[i]] = y
    err = model.predict(data)[0] - constant
    curr_error = abs(err)

    # df: pd.DataFrame = pd.read_pickle('exports/ml_data.pickle')
    # ml_data = pd.DataFrame([{'x': x, 'to_opt': to_opt, 'data': data, 'constant': constant, 'err': err}])
    # ml_data['timestep'] = timestep
    # try:
    #     ml_data['iteration'] = df.iloc[-1]['iteration'] + 1
    # except IndexError:
    #     ml_data['iteration'] = 0
    # df.append(ml_data, ignore_index=True).to_pickle('exports/ml_data.pickle')

    #print(err)
    return curr_error

# Model Configuration

In [11]:
eth_price = pd.DataFrame(debt_market_df['rho_star'])
eth_p_mean = np.mean(eth_price.to_numpy().flatten())

mar_price = pd.DataFrame(debt_market_df['p'])
mar_p_mean = np.mean(mar_price.to_numpy().flatten())

eth_returns = ((eth_price - eth_price.shift(1))/eth_price.shift(1)).to_numpy().flatten()
eth_gross_returns = (eth_price / eth_price.shift(1)).to_numpy().flatten()

eth_returns_mean = np.mean(eth_returns[1:])

eth_p_mean, eth_returns_mean, mar_p_mean

(365.8127290676168, -0.0012148823654996293, 1.0004125645956772)

In [12]:
#eth_collateral = historical_initial_state['Q'] / genesis_cdp_count # collateral per genesis CDP

eth_price_ = eth_price.loc[start_date][0]

liquidation_ratio = 1.5 # 150%
liquidation_buffer = 2.0
#collateral_value = eth_collateral * eth_price_
target_price = 1.0
# principal_debt = collateral_value / (target_price * liquidation_ratio * liquidation_buffer)
#principal_debt = historical_initial_state['D_1'] / genesis_cdp_count # debt per genesis CDP

#collateralization_ratio = collateral_value / principal_debt * target_price

# print(f'''
# Initial ETH price: {eth_price_}
# Initial RAI price: {target_price}
# Initial collateralization ratio (ratio + buffer): {collateralization_ratio}
# Initial debt value: {principal_debt * target_price}
# Initial collateral value: {collateral_value}
# ''')

In [13]:
stability_fee = (historical_initial_state['beta'] * 30 / 365) / (30 * 24 * 3600)

In [14]:
partial_results = pd.DataFrame()
partial_results_file = 'exports/partial_results.pickle'
partial_results.to_pickle(partial_results_file)

ml_data = pd.DataFrame()
ml_data_file = 'exports/ml_data.pickle'
ml_data.to_pickle(ml_data_file)

In [15]:
def save_simulation(name, initial_state, params, results_df):
    import dill as pickle
    import os
    os.makedirs(f'exports/simulation_results/{name}', exist_ok=True)
    with open(f'exports/simulation_results/{name}/initial_state.pickle', 'wb') as f:
        pickle.dump(initial_state, f, pickle.HIGHEST_PROTOCOL)
    with open(f'exports/simulation_results/{name}/params.pickle', 'wb') as f:
        pickle.dump(params, f, pickle.HIGHEST_PROTOCOL)
    results_df.to_csv(f'exports/simulation_results/{name}/results.csv')

import json

def optimal_results_from_json(row):
    row['optimal_values'] = json.loads(row['optimal_values'].replace("'", "\""))
    return row

In [16]:
simulation_results_df = pd.read_pickle('simulations/debt_market/results_desktop/results/2020-12-07 23:39:24.524707/results.pickle')

In [17]:
import math

eth_collateral = historical_initial_state['Q']
eth_locked = debt_market_df.loc[:start_date]['v_1'].sum()
eth_freed = debt_market_df.loc[:start_date]['v_2 + v_3'].sum() / 2
eth_bitten = debt_market_df.loc[:start_date]['v_2 + v_3'].sum() / 2

print(f'''
{eth_collateral}
{eth_locked}
{eth_freed}
{eth_bitten}
''')

assert math.isclose(eth_collateral, eth_locked - eth_freed - eth_bitten, abs_tol=1e-6)

principal_debt = historical_initial_state['D_1']
rai_drawn = debt_market_df.loc[:start_date]['u_1'].sum()
rai_wiped = debt_market_df.loc[:start_date]['u_2'].sum()
rai_bitten = debt_market_df.loc[:start_date]['u_3'].sum()

print(f'''
{principal_debt}
{rai_drawn}
{rai_wiped}
{rai_bitten}
''')

assert math.isclose(principal_debt, rai_drawn - rai_wiped - rai_bitten, abs_tol=1e-6)

print(f'Collateralization ratio: {eth_collateral * eth_price_ / principal_debt * target_price}')


947045.6487308872
1283927.878365983
168441.11481754784
168441.11481754784


68492685.99189329
153866330.33664766
56269497.29762668
29104147.047127675

Collateralization ratio: 2.884101718822069


In [18]:
# At historical start date:
median_cdp_size = 2500 # dollars
average_cdp_size = 50 # dollars
genesis_cdp_count = int(eth_collateral / median_cdp_size)
genesis_cdp_count

18940

In [19]:
# Create a "genesis" CDP

cdp_list = []
for i in range(genesis_cdp_count):
    cdp_list.append({
        'open': 1, # True/False == 1/0 for integer/float series
        'time': 0,
        'locked': eth_collateral / genesis_cdp_count,
        'drawn': principal_debt / genesis_cdp_count,
        'wiped': 0.0,
        'freed': 0.0,
        'w_wiped': 0.0,
        'v_bitten': 0.0,
        'u_bitten': 0.0,
        'w_bitten': 0.0,
        'dripped': 0.0
    })

# for i in range(genesis_cdp_count):
#     cdp_list.append({
#         'open': 1, # True/False == 1/0 for integer/float series
#         'time': 0,
#         'locked': historical_initial_state['v_1'],
#         'drawn': historical_initial_state['u_1'],
#         'wiped': historical_initial_state['u_2'],
#         'freed': 0.0,
#         'w_wiped': historical_initial_state['w_2'],
#         'v_bitten': historical_initial_state['v_2 + v_3'],
#         'u_bitten': historical_initial_state['u_3'],
#         'w_bitten': historical_initial_state['w_3'],
#         'dripped': 0.0
#     })

cdps = pd.DataFrame(cdp_list)
cdps

,open,time,locked,drawn,wiped,freed,w_wiped,v_bitten,u_bitten,w_bitten,dripped
0,1,0,50.00241,3616.298099,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,50.00241,3616.298099,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,50.00241,3616.298099,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,50.00241,3616.298099,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,50.00241,3616.298099,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
18935,1,0,50.00241,3616.298099,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18936,1,0,50.00241,3616.298099,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18937,1,0,50.00241,3616.298099,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18938,1,0,50.00241,3616.298099,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
market_price = debt_market_df.loc[start_date]['p']
market_price

0.989559416140834

In [21]:
initial_state = {
    'events': [],
    'cdps': cdps,
    # Loaded from exogenous parameter
    'eth_price': eth_price_, # dollars
    # v
    'eth_collateral': eth_collateral, # Q
    'eth_locked': eth_locked, # v1
    'eth_freed': eth_freed, # v2
    'eth_bitten': eth_bitten, # v3
    'v_1': historical_initial_state['v_1'],
    'v_2': historical_initial_state['v_2 + v_3'] / 2,
    'v_3': historical_initial_state['v_2 + v_3'] / 2,
    # u
    'principal_debt': principal_debt, # D1
    'rai_drawn': rai_drawn, # u1 "minted"
    'rai_wiped': rai_wiped, # u2
    'rai_bitten': rai_bitten, # u3
    'u_1': historical_initial_state['u_1'],
    'u_2': historical_initial_state['u_2'],
    'u_3': historical_initial_state['u_3'],
    # w
    'w_1': historical_initial_state['w_1'],
    'w_2': historical_initial_state['w_2'],
    'w_3': historical_initial_state['w_3'],
    'accrued_interest': historical_initial_state['D_2'],
    'stability_fee': stability_fee,
    'market_price': market_price,
    'target_price': target_price, # dollars == redemption price
    'target_rate': 0 / (30 * 24 * 3600), # per second interest rate (X% per month)
    'p_expected': target_price,
    'p_debt_expected': target_price,
}

# initial_state = {
#     'events': [],
#     'cdps': cdps,
#     # Loaded from exogenous parameter
#     'eth_price': eth_price.iloc[0], # dollars
#     # v
#     'eth_collateral': historical_initial_state['Q'] * genesis_cdp_count, # Q
#     'eth_locked': historical_initial_state['v_1'] * genesis_cdp_count, # v1
#     'eth_freed': 0.0 * genesis_cdp_count, # v2
#     'eth_bitten': historical_initial_state['v_2 + v_3'] * genesis_cdp_count, # v3
#     'v_1': historical_initial_state['v_1'],
#     'v_2': 0.0,
#     'v_3': historical_initial_state['v_2 + v_3'],
#     # u
#     'principal_debt': historical_initial_state['D_1'] * genesis_cdp_count, # D1
#     'rai_drawn': historical_initial_state['u_1'] * genesis_cdp_count, # u1 "minted"
#     'rai_wiped': historical_initial_state['u_2'] * genesis_cdp_count, # u2
#     'rai_bitten': historical_initial_state['u_3'] * genesis_cdp_count, # u3
#     'u_1': historical_initial_state['u_1'],
#     'u_2': historical_initial_state['u_2'],
#     'u_3': historical_initial_state['u_3'],
#     # w
#     'w_1': historical_initial_state['w_1'],
#     'w_2': historical_initial_state['w_2'],
#     'w_3': historical_initial_state['w_3'],
#     'accrued_interest': historical_initial_state['D_2'] * genesis_cdp_count,
#     'stability_fee': stability_fee,
#     'market_price': debt_market_df.iloc[0]['p'],
#     'target_price': target_price, # dollars == redemption price
#     'target_rate': 0 / (30 * 24 * 3600), # per second interest rate (X% per month)
#     'p_expected': target_price,
#     'p_debt_expected': target_price,
# }

initial_state.update(historical_initial_state)

parameters = {
    'debug': [True], # Print debug messages (see APT model)
    'raise_on_assert': [False], # See assert_log() in utils.py
    'test': [
        # {
        #     'enable': False
        # },
        {
            'enable': True,
            'params': {
                'optimal_values': {
                    'v_1': lambda timestep=0, df=simulation_results_df: \
                        simulation_results_df.iloc[timestep - 1]['optimal_values'].get('v_1', historical_initial_state['v_1']),
                    'v_2 + v_3': lambda timestep=0, df=simulation_results_df: \
                        simulation_results_df.iloc[timestep - 1]['optimal_values'].get('v_2 + v_3', historical_initial_state['v_2 + v_3']),
                    'u_1': lambda timestep=0, df=simulation_results_df: \
                        simulation_results_df.iloc[timestep - 1]['optimal_values'].get('u_1', historical_initial_state['u_1']),
                    'u_2': lambda timestep=0, df=simulation_results_df: \
                        simulation_results_df.iloc[timestep - 1]['optimal_values'].get('u_2', historical_initial_state['u_2'])
                }
            }
        },
        # {
        #     'enable': False,
        #     'params': {
        #         'optimal_values': {
        #             'v_1': lambda timestep=0: historical_initial_state['v_1'],
        #             'v_2 + v_3': lambda timestep=0: historical_initial_state['v_2 + v_3'],
        #             'u_1': lambda timestep=0: historical_initial_state['u_1'],
        #             'u_2': lambda timestep=0: historical_initial_state['u_2']
        #         }
        #     }
        # },
        # {
        #     'enable': True,
        #     'params': {
        #         'optimal_values': {
        #             'v_1': lambda timestep=0: 1000,
        #             'v_2 + v_3': lambda timestep=0: 500,
        #             'u_1': lambda timestep=0: 100,
        #             'u_2': lambda timestep=0: 50
        #         }
        #     }
        # },
        # {
        #     'enable': True,
        #     'params': {
        #         'optimal_values': {
        #             'v_1': lambda timestep=0: 500,
        #             'v_2 + v_3': lambda timestep=0: 1000,
        #             'u_1': lambda timestep=0: 50,
        #             'u_2': lambda timestep=0: 100
        #         }
        #     }
        # }
    ],
    'free_memory_states': [['cdps', 'events']], #'cdps',
    #'eth_market_std': [1],
    #'random_state': [np.random.RandomState(seed=0)],
    'liquidation_ratio': [liquidation_ratio], # %
    'liquidation_buffer': [liquidation_buffer], # multiplier applied to CDP collateral by users
    'stability_fee': [lambda timestep, df=debt_market_df: stability_fee], # df.iloc[timestep].beta / (365 * 24 * 3600), # per second interest rate (1.5% per month)
    'liquidation_penalty': [0], # 0.13 == 13%
    'cdp_top_up_buffer': [2],
    # Average CDP duration == 3 months: https://www.placeholder.vc/blog/2019/3/1/maker-network-report
    # The tuning of this parameter is probably off the average, because we don't have the CDP size distribution matched yet,
    # so although the individual CDPs could have an average debt age of 3 months, the larger CDPs likely had a longer debt age.
    'average_debt_age': [3 * (30 * 24 * 3600)], # delta t (seconds)
    'eth_price': [lambda timestep, df=debt_market_df: df.iloc[timestep].rho_star],
    #'v_1': [lambda state, _state_history, df=debt_market_df: df.iloc[state['timestep']].v_1], # Driven by historical data
    #'u_1': [lambda timestep, df=debt_market_df: df.iloc[timestep].u_1], # Driven by historical data
    'seconds_passed': [lambda timestep, df=debt_market_df: df.iloc[timestep].seconds_passed],
    # 'market_price': [lambda timestep, df=debt_market_df: target_price],
    # APT model
    # **{
    #     'use_APT_ML_model': [False],
    #     'root_function': [G_OLS], # glf, G, G_OLS
    #     'features': [features], # features_ml, features
    # },
    **{
        'use_APT_ML_model': [True],
        'root_function': [glf], # glf, G, G_OLS
        'callback': [glf_continue_callback], # glf callback
        'model': [model],
        'features': [features_ml], # features_ml, features
    },
    'freeze_feature_vector': [False], # Use the same initial state as the feature vector for each timestep
    'optvars': [optvars],
    'bounds': [[(xmin,debt_market_df[optvars].max()[i]) 
        for i,xmin in enumerate(debt_market_df[optvars].min())
    ]],
    'interest_rate': [1.0],
    'eth_p_mean': [eth_p_mean],
    'eth_returns_mean': [eth_returns_mean],
    'mar_p_mean': [mar_p_mean],
    # APT OLS model
    'alpha_0': [0],
    'alpha_1': [1],
    'beta_0': [1.0003953223600617],
    'beta_1': [0.6756295152422528],
    'beta_2': [3.86810578185312e-06],    
    # Controller
    'controller_enabled': [True],
    'kp': [-1.5e-6], #5e-7 #proportional term for the stability controller: units 1/USD
    'ki': [lambda control_period=3600: 0 / control_period], #-1e-7 #integral term for the stability controller: units 1/(USD*seconds)
    'partial_results': [partial_results_file],
}

# parameters = parameters.update({
#     'delta_v1': [lambda state, state_history: delta_v1(state, state_history)],
#     'market_price': [lambda timestep, df=debt_market_df: df.iloc[timestep].p]
# })

# Simulation Execution

In [22]:
SIMULATION_TIMESTEPS = len(debt_market_df) - 1 # approx. 600
MONTE_CARLO_RUNS = 1

In [23]:
from models.config_wrapper import ConfigWrapper
import models.system_model_v2 as system_model_v2

system_simulation = ConfigWrapper(system_model_v2, T=range(SIMULATION_TIMESTEPS), M=parameters, initial_state=initial_state)

In [24]:
from cadCAD import configs
del configs[:]

system_simulation.append()

(simulation_result, tensor_field, sessions) = run(drop_midsteps=True)

INFO:root:Started simulation
DEBUG:root:p_expected terms: (1, 0.989559416140834, 1.0, 3.86810578185312e-06, 365.8127290676168, 208.585376703518, 0.6756295152422528, 1.0004125645956772, 0, 0.9974173881830576)
DEBUG:root:
    ##### APT model run #####
    Timestep: 1
    
INFO:root:APT test enabled

                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (651, 53, 1, 48)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
DEBUG:root:resolve_cdp_positions_unified() ~ Number of open CDPs: 18951; Number of closed CDPs: 0
DEBUG:root:1 CDPs liquidated with v_2 0.0 v_3 428.0880701044807 u_3 365451.38671366364 w_3 0.0
DEBUG:root:p_rebalance_cdps() ~ Number of

KeyboardInterrupt: 

In [24]:
partial_results: pd.DataFrame = pd.read_pickle(partial_results_file)
partial_results

,D,D_1,D_2,Q,accrued_interest,beta,blockheight,cdps,cumulative_time,error_star,...,u_2 + u_3,u_3,v_1,v_2,v_2 + v_3,v_3,w_1,w_2,w_2 + w_3,w_3
0,6.878286e+07,6.849269e+07,290170.723889,947045.648731,294882.042983,0.025,0.0,open time locked drawn...,86400.0,0.010441,...,84787.982838,0.0,1000.000000,500.000000,1000.505698,0.000000e+00,4691.447132,0.000000e+00,22.312545,0.0
1,6.878286e+07,6.849269e+07,290170.723889,947045.648731,299593.688206,0.025,0.0,open time locked draw...,172800.0,0.008171,...,84787.982838,0.0,1000.000000,500.000000,1000.505698,0.000000e+00,2.705760,4.691447e+03,22.312545,0.0
2,6.878286e+07,6.849269e+07,290170.723889,947045.648731,304305.659581,0.025,0.0,open time locked draw...,259200.0,0.001833,...,84787.982838,0.0,1000.000000,500.000000,1000.505698,0.000000e+00,13539.102900,0.000000e+00,22.312545,0.0
3,6.878286e+07,6.849269e+07,290170.723889,947045.648731,309017.957128,0.025,0.0,open time locked draw...,345600.0,-0.000761,...,84787.982838,0.0,1000.000000,500.000000,1000.505698,0.000000e+00,10.202084,1.354181e+04,22.312545,0.0
4,6.878286e+07,6.849269e+07,290170.723889,947045.648731,313730.580872,0.025,0.0,open time locked draw...,432000.0,0.005090,...,84787.982838,0.0,1000.000000,500.000000,1000.505698,0.000000e+00,592.859444,-3.637979e-12,22.312545,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,6.878286e+07,6.849269e+07,290170.723889,947045.648731,292536.900536,0.025,0.0,open time locked d...,8294400.0,-0.040011,...,84787.982838,0.0,129180.736297,99318.387615,1000.505698,0.000000e+00,1573.115511,0.000000e+00,22.312545,0.0
68,6.878286e+07,6.849269e+07,290170.723889,947045.648731,292556.937995,0.025,0.0,open time locked d...,8380800.0,0.011907,...,84787.982838,0.0,166187.408058,2032.505520,1000.505698,0.000000e+00,836.450827,9.702798e+03,22.312545,0.0
69,6.878286e+07,6.849269e+07,290170.723889,947045.648731,292576.976827,0.025,0.0,open time locked d...,8467200.0,-0.033904,...,84787.982838,0.0,166205.900276,4268.330984,1000.505698,-3.637979e-12,5069.244567,0.000000e+00,22.312545,0.0
70,6.878286e+07,6.849269e+07,290170.723889,947045.648731,292597.017031,0.025,0.0,open time locked dr...,8553600.0,-0.033741,...,84787.982838,0.0,49826.454154,30938.310842,1000.505698,0.000000e+00,1394.753096,0.000000e+00,22.312545,0.0


In [25]:
partial_results.plot(x='timestamp', y=['eth_collateral', 'principal_debt'])

In [26]:
# ml_data: pd.DataFrame = pd.read_pickle(ml_data_file)
# ml_data

In [27]:

# # ml_data.query('timestep == 1').plot(x='iteration', y='err_abs')

# import plotly.express as px
# ml_data: pd.DataFrame = pd.read_pickle(ml_data_file)
# ml_data['err_abs'] = ml_data.err.abs()
# ml_data = ml_data.query('timestep == 1')
# fig = px.line(ml_data, x="iteration", y="err_abs", facet_col="timestep", facet_col_wrap=2, log_y=True)
# fig.update_yaxes(matches=None)
# fig.update_xaxes(matches=None)
# fig.show()

In [28]:
# Print system events e.g. liquidation assertion errors
simulation_result[simulation_result.events.astype(bool)].events.apply(lambda x: x[0])

Series([], Name: events, dtype: object)

# Simulation Analysis

In [29]:
#simulation_result = pd.concat([simulation_result, debt_market_df.reset_index()], axis=1)

simulation_result = simulation_result.assign(eth_collateral_value = simulation_result.eth_collateral * simulation_result.eth_price)

simulation_result['collateralization_ratio'] = (simulation_result.eth_collateral * simulation_result.eth_price) / (simulation_result.principal_debt * simulation_result.target_price)
#simulation_result['historical_collateralization_ratio'] = (simulation_result.Q * simulation_result.rho_star) / (simulation_result.D_1 * simulation_result.p)

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

simulation_result

,index,events,error_star,error_star_integral,market_price,blockheight,timedelta,cumulative_time,timestamp,cdps,eth_price,eth_collateral,eth_locked,eth_freed,eth_bitten,v_1,v_2,v_3,principal_debt,rai_drawn,rai_wiped,rai_bitten,u_1,u_2,u_3,accrued_interest,system_revenue,stability_fee,interest_dripped,interest_wiped,interest_bitten,w_1,w_2,w_3,target_price,target_rate,p_expected,eth_return,eth_gross_return,optimal_values,p_debt_expected,beta,Q,v_2 + v_3,D_1,u_2 + u_3,D_2,w_2 + w_3,D,simulation,subset,run,substep,timestep,eth_collateral_value,collateralization_ratio
0,0,None,0.000000,0.000000,0.989559,0,0.0,0.0,2017-12-18,None,208.585377,9.470456e+05,1.283928e+06,1.684411e+05,168441.114818,2857.187535,500.252849,5.002528e+02,6.849269e+07,1.538663e+08,5.626950e+07,2.910415e+07,4.647793e+05,8.478798e+04,8.876668e-10,290170.723889,0.000000e+00,7.927448e-10,0,0,0,4653.072701,22.312545,2.335957e-13,1.000000,0.000000e+00,1.000000,0.000000,0.0,{},1.000000,0.025,947045.648731,1000.505698,6.849269e+07,84787.982838,290170.723889,22.312545,6.878286e+07,0,0,1,0,0,1.975399e+08,2.884102
1,20,None,0.010441,451.033223,0.992935,0,86400.0,86400.0,2017-12-19,None,832.236972,1.105474e+06,1.450133e+06,1.762183e+05,168441.114818,166205.424364,7777.152801,0.000000e+00,6.773081e+07,1.547520e+08,5.791707e+07,2.910415e+07,1.108562e+07,2.612333e+06,0.000000e+00,294829.854240,0.000000e+00,7.927448e-10,0,0,0,4752.106459,0.000000,0.000000e+00,0.998648,-1.566088e-08,0.997417,2.989910,1.0,"{'u_1': 885695.8864781586, 'u_2': 1647575.4769...",0.993199,0.025,947045.648731,1000.505698,6.849269e+07,84787.982838,290170.723889,22.312545,6.878286e+07,0,0,1,20,1,9.200163e+08,13.601816
2,40,None,0.005713,1105.843552,0.994378,0,86400.0,172800.0,2017-12-20,None,810.287507,1.128419e+06,1.482041e+06,1.851804e+05,168441.114818,31907.424056,8962.118715,0.000000e+00,7.001570e+07,1.586835e+08,5.956368e+07,2.910415e+07,2.383741e+08,1.646609e+06,0.000000e+00,299645.808731,4.752106e+03,7.927448e-10,0,0,0,968.264413,4752.106459,0.000000e+00,0.997909,-8.569803e-09,0.996160,-0.026374,1.0,"{'u_1': 3931503.8888005996, 'u_2': 1646608.879...",0.998852,0.025,947045.648731,1000.505698,6.849269e+07,84787.982838,290170.723889,22.312545,6.878286e+07,0,0,1,20,2,9.143440e+08,13.086496
3,60,None,0.003531,1400.351738,1.003992,0,86400.0,259200.0,2017-12-21,None,837.231080,1.164616e+06,1.522987e+06,1.899304e+05,168441.114818,40946.645785,4750.045273,0.000000e+00,7.641973e+07,1.658054e+08,6.028157e+07,2.910415e+07,9.813630e+06,8.145674e+06,0.000000e+00,304900.740325,4.752106e+03,7.927448e-10,0,0,0,16546.145228,0.000000,0.000000e+00,0.997452,-5.296577e-09,0.996722,0.033252,1.0,"{'u_1': 7121917.083213594, 'u_2': 717886.48361...",1.004149,0.025,947045.648731,1000.505698,6.849269e+07,84787.982838,290170.723889,22.312545,6.878286e+07,0,0,1,20,3,9.750526e+08,12.791765
4,80,None,-0.006540,1138.022233,0.987360,0,86400.0,345600.0,2017-12-22,None,689.014990,1.079314e+06,1.535024e+06,2.872687e+05,168441.114818,12036.783556,97338.253349,1.818989e-12,7.015743e+07,1.658132e+08,6.655165e+07,4.230218e+07,8.306732e+06,8.744675e+06,1.319803e+07,308798.575568,3.107288e+04,7.927448e-10,0,0,0,2.549801,17514.409641,9.284535e+02,0.998298,9.809694e-09,0.999733,-0.177031,1.0,"{'u_1': 7785.08774378337, 'u_2': 6270083.58551...",0.976572,0.025,947045.648731,1000.505698,6.849269e+07,84787.982838,290170.723889,22.312545,6.878286e+07,0,0,1,20,4,7.436638e+08,10.618002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1920,None,-0.040011,-5616.821935,1.013807,0,86400.0,8294400.0,2018-03-24,None,537.890313,8.519854e+06,1.283471e+07,4.126849e+06,188002.355396,129180.736297,99318.387615,0.000000e+00,0.000000e+00,6.054355e+08,4.169928e+08,3.972862e+08,5.091426e+06,7.720823e+07,0.000000e+00,292536.900536,5.495903e+06,7.927448e-10,0,0,0,1573.115511,0.000000,0.00

## Save simulation

In [30]:
# from datetime import datetime
# name = datetime.now()
# save_simulation(name, initial_state, parameters, simulation_result)

In [31]:
# simulation_result = pd.read_csv('exports/simulation_results/jamsheed/simulation_result-2020-12-07 13_08_48.880831.csv')
# simulation_result

## Select simulation

In [32]:
df = simulation_result.query('simulation == 0 and subset == 0')

## Historical ETH price: December 2017 to September 2019

In [33]:
df.plot(x='timestamp', y=['eth_price'])

In [34]:
df.plot(x='timestamp', y=['eth_return'])

## Target price / redemption price set to 1 "dollar" for historical comparison

In [35]:
df.plot(x='timestamp', y=['target_price', 'market_price'])

In [36]:
df.plot(x='timestamp', y=['p_expected', 'p_debt_expected'])

In [37]:
df.plot(x='timestamp', y=['target_rate'])

## Historical system ETH collateral vs. model

In [38]:
df['locked - freed - bitten'] = df['eth_locked'] - df['eth_freed'] - df['eth_bitten']
df.plot(y=['eth_collateral', 'locked - freed - bitten']) #'Q'

## Historical system ETH collateral value vs. model

In [39]:
df.plot(x='timestamp', y=['eth_collateral_value']) #'C_star'

## Debt market ETH activity

In [40]:
df.plot(x='timestamp', y=['eth_locked', 'eth_freed', 'eth_bitten'])

In [41]:
df.plot(x='timestamp', y=['v_1', 'v_2', 'v_3'])

## Debt market principal debt "Rai" activity

In [42]:
df['drawn - wiped - bitten'] = df['rai_drawn'] - df['rai_wiped'] - df['rai_bitten']
df.plot(x='timestamp', y=['principal_debt', 'drawn - wiped - bitten']) #, 'D_1'

In [43]:
df.plot(x='timestamp', y=['rai_drawn', 'rai_wiped', 'rai_bitten'])

In [44]:
df.plot(x='timestamp', y=['u_1', 'u_2', 'u_3'])

## Accrued interest and system revenue (MKR)

In [45]:
df.plot(x='timestamp', y=['w_1', 'w_2', 'w_3'])

In [46]:
df.plot(x='timestamp', y=['accrued_interest']) #, 'D_2'

In [47]:
df.plot(x='timestamp', y=['system_revenue'])

## Historical collateralization ratio vs. model

In [48]:
df.plot(x='timestamp', y=['collateralization_ratio']) #, 'historical_collateralization_ratio'